<a href="https://colab.research.google.com/github/ProtossDragoon/paper_implementation_and_testing_tf2/blob/main/utils/Generate_TFRecord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate TFRecord

## Author

name : Janghoo Lee <br>
github : https://github.com/ProtossDragoon <br>
contact : dlwkdgn1@naver.com <br>
circle : https://github.com/sju-coml <br>
organization : https://web.deering.co/ <br>
published date : June, 2021


## Related Notebook

[Notebooks](https://github.com/ProtossDragoon/paper_implementation_and_testing_tf2/tree/main/notebooks)


# Environment

In [112]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [113]:
import os, sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm.notebook import tqdm as tqdm
import cv2

## Google Drive / Git

In [114]:
#!rm -r /content/gdrive/
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [127]:
USE_GCS = False
HOME_DIR = "/content/gdrive/MyDrive"
WS = os.path.join(HOME_DIR, "ColabWorkspace")
DATA_DIR = os.path.join(HOME_DIR, 'data')
DATA_INDEXFILE_DIR = os.path.join(DATA_DIR, 'index')
DATA_TFRECORD_DIR = os.path.join(DATA_DIR, 'tfrecord')

GIT_REPO_NAME = 'paper_implementation_and_testing_tf2'
GIT_WS = os.path.join(WS, GIT_REPO_NAME)

GIT_BRANCH = 'main'
GIT_USERNAME = None # Add herea
GIT_EMAIL = None
GIT_PASSWORD = None # Add here

In [128]:
!mkdir -p {WS}
%cd {WS}
!git clone https://github.com/ProtossDragoon/{GIT_REPO_NAME}.git
%cd {GIT_WS}
!git config --global user.name {GIT_USERNAME}
!git config --global user.email {GIT_EMAIL}

!git checkout -b {GIT_BRANCH}
!git checkout {GIT_BRANCH}
!git pull origin {GIT_BRANCH}
!git branch
!git branch --set-upstream-to origin/{GIT_BRANCH}

/content/gdrive/MyDrive/ColabWorkspace
fatal: destination path 'paper_implementation_and_testing_tf2' already exists and is not an empty directory.
/content/gdrive/MyDrive/ColabWorkspace/paper_implementation_and_testing_tf2
fatal: A branch named 'main' already exists.
D	utils/Generate_TFRecord.ipynb
D	utils/rosutils/Convert_Bagfile_to_Video.ipynb
Already on 'main'
Your branch is up to date with 'origin/main'.
From https://github.com/ProtossDragoon/paper_implementation_and_testing_tf2
 * branch            main       -> FETCH_HEAD
Already up to date.
  list
* main
  resnet
  tftrt
  unet
Branch 'main' set up to track remote branch 'main' from 'origin'.


## [Additional] GCP & TFCloud

Only run for GCP & TF-Cloud. Not for Vanilla google drive user.

### Authenticate GCP User

In [130]:
# !pip install tensorflow_cloud # google colab already has tensorflow cloud
!pip install tensorflow_cloud -q
import tensorflow_cloud as tfc
print(tfc.__version__)

GCP_BUCKET_NAME = "deer-rudolph" #@param {type:"string"}
GCP_PROJECT_NAME = "deer-deep-learning-project" #@param {type:"string"}
GCP_PROJECT_ID = "linear-freehold-314804" #@param {type:"string"}

IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
    if "google.colab" in sys.modules:
        from google.colab import auth
        auth.authenticate_user()
        os.environ["GOOGLE_CLOUD_PROJECT"] = GCP_PROJECT_ID

USE_GCS = True

     |████████████████████████████████| 102kB 3.7MB/s 
     |████████████████████████████████| 409kB 7.5MB/s 
     |████████████████████████████████| 102kB 4.8MB/s 
     |████████████████████████████████| 153kB 8.3MB/s 
     |████████████████████████████████| 17.7MB 323kB/s 
     |████████████████████████████████| 1.0MB 35.0MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 19.0MB 206kB/s 
     |████████████████████████████████| 9.6MB 34.1MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
     |████████████████████████████████| 2.3MB 29.6MB/s 
     |████████████████████████████████| 256kB 43.4MB/s 
     |████████████████████████████████| 174kB 47.6MB/s 
     |████████████████████████████████| 276kB 47.7MB/s 
     |████████████████████████████████| 174kB 46.8MB/s 
     |████████████████████████████████| 184kB 44.0MB/s 
     |████████████████████████████████| 440kB 45.5MB/s 
     |████████████████████████████████| 92kB 7.9MB/s 
  

### Global Hyper Parameters

Re-define hyper parameters we defined before.

In [131]:
HOME_DIR = os.path.join('gs://', GCP_BUCKET_NAME)
DATA_DIR = os.path.join(HOME_DIR, 'data')
DATA_INDEXFILE_DIR = os.path.join(DATA_DIR, 'index')
DATA_TFRECORD_DIR = os.path.join(DATA_DIR, 'tfrecord')

# Dataset Library

## CamVid


For this example we will use CamVid dataset. It is a set of:
- **train** images + segmentation masks
- **validation** images + segmentation masks
- **test** images + segmentation masks
- All images have 320 pixels height and 480 pixels width.

<br>

For more inforamtion about dataset visit 
- http://www0.cs.ucl.ac.uk/staff/G.Brostow/papers/Brostow_2009-PRL.pdf
- http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamVid/

### Segmentation task

In [132]:
DATASET_DIR_CAMVID = os.path.join(DATA_DIR, 'camvid')
DATASET_TRAIN_X_DIR_CAMVID = os.path.join(DATASET_DIR_CAMVID, 'train')
DATASET_TRAIN_Y_DIR_CAMVID = os.path.join(DATASET_DIR_CAMVID, 'trainannot')
DATASET_VAL_X_DIR_CAMVID = os.path.join(DATASET_DIR_CAMVID, 'val')
DATASET_VAL_Y_DIR_CAMVID = os.path.join(DATASET_DIR_CAMVID, 'valannot')
DATASET_TEST_X_DIR_CAMVID = os.path.join(DATASET_DIR_CAMVID, 'test')
DATASET_TEST_Y_DIR_CAMVID = os.path.join(DATASET_DIR_CAMVID, 'testannot')

print('parsed : {}'.format(DATASET_DIR_CAMVID))
if USE_GCS:
    !gsutil ls {DATASET_DIR_CAMVID}
else:
    %cd {DATASET_DIR_CAMVID}
    %ls -al

parsed : gs://deer-rudolph/data/camvid
gs://deer-rudolph/data/camvid/test.txt
gs://deer-rudolph/data/camvid/train.txt
gs://deer-rudolph/data/camvid/val.txt
gs://deer-rudolph/data/camvid/test/
gs://deer-rudolph/data/camvid/testannot/
gs://deer-rudolph/data/camvid/train/
gs://deer-rudolph/data/camvid/trainannot/
gs://deer-rudolph/data/camvid/val/
gs://deer-rudolph/data/camvid/valannot/


## Aihub Pedestrian

For this example we will use Aihub Pedestrian dataset. It is a set of:
- **images and labels** images + segmentation masks
- All images have 1080 pixels height and 1920 pixels width.

<br>

For more inforamtion about dataset visit 
- https://www.aihub.or.kr/aidata/136

<br>

This notebook does not have dataset download link.

### Segmentation task

In [133]:
DATASET_DIR_PEDESTRIAN = os.path.join(DATA_DIR, 'aihubpedestrian')

print('parsed : {}'.format(DATASET_DIR_PEDESTRIAN))
if USE_GCS:
    !gsutil ls {DATASET_DIR_PEDESTRIAN}
else:
    %cd {DATASET_DIR_PEDESTRIAN}
    %ls -al

parsed : gs://deer-rudolph/data/aihubpedestrian
CommandException: One or more URLs matched no objects.


# Convert

## Global Hyper Parameters

In [134]:
DATASETNAME = "camvid" #@param ['aihubpedestrian', 'camvid']
TXT_FILE_DIR = os.path.join(DATA_INDEXFILE_DIR, DATASETNAME.upper())
TFRECORD_FILE_DIR = os.path.join(DATA_TFRECORD_DIR, DATASETNAME.upper())
TFRECORD_FROM_INDEXFILE = True #@param {type:"boolean"}

## Generate Index file

### Hyper Parameters

In [135]:
SEED = 10 #@param {type:"integer"}

print("Indexfile generation result save  at {}".format(TXT_FILE_DIR))
print("See {}".format(TXT_FILE_DIR))
if USE_GCS:
    !gsutil ls {TXT_FILE_DIR}
else:
    !mkdir -p {TXT_FILE_DIR}
    %cd {TXT_FILE_DIR}
    %ls -al

Indexfile generation result save  at gs://deer-rudolph/data/index/CAMVID
See gs://deer-rudolph/data/index/CAMVID
gs://deer-rudolph/data/index/CAMVID/
gs://deer-rudolph/data/index/CAMVID/test
gs://deer-rudolph/data/index/CAMVID/train
gs://deer-rudolph/data/index/CAMVID/val


### Run

In [64]:
def generate_index_camvid_segmentation(data_dir, 
                                       train_dir, 
                                       trainannot_dir, 
                                       val_dir, 
                                       valannot_dir, 
                                       test_dir, 
                                       testannot_dir, 
                                       save_dir):
    assert tf.io.gfile.isdir(save_dir), '{} is not a directory!'.format(save_dir)
    assert tf.io.gfile.isdir(train_dir), '{} is not a directory!'.format(train_dir)
    assert tf.io.gfile.isdir(trainannot_dir), '{} is not a directory!'.format(trainannot_dir)
    assert tf.io.gfile.isdir(val_dir), '{} is not a directory!'.format(val_dir)
    assert tf.io.gfile.isdir(valannot_dir), '{} is not a directory!'.format(valannot_dir)
    assert tf.io.gfile.isdir(test_dir), '{} is not a directory!'.format(test_dir)
    assert tf.io.gfile.isdir(testannot_dir), '{} is not a directory!'.format(testannot_dir)
    print('generate 3 files :\n{}\n{}\n{}\n'.format(os.path.join(save_dir, 'train.txt'),
                                                    os.path.join(save_dir, 'val.txt'),
                                                    os.path.join(save_dir, 'test.txt'),))
    
    image_train_name_li = tf.io.gfile.listdir(train_dir)
    mask_train_name_li = tf.io.gfile.listdir(trainannot_dir)
    image_val_name_li = tf.io.gfile.listdir(val_dir)
    mask_val_name_li = tf.io.gfile.listdir(valannot_dir)
    image_test_name_li = tf.io.gfile.listdir(test_dir)
    mask_test_name_li = tf.io.gfile.listdir(testannot_dir)
    
    print('Number of image file {}'.format(len(image_train_name_li)+len(image_val_name_li)+len(image_test_name_li)))
    print('Number of mask file {}'.format(len(mask_train_name_li)+len(mask_val_name_li)+len(mask_test_name_li)))

    tf.io.gfile.makedirs(save_dir)
    for f_name, (image_dir, mask_dir), (image_file_names, mask_file_names) in zip(['train', 'val', 'test'], 
                                                                                  [(train_dir, trainannot_dir),
                                                                                   (val_dir, valannot_dir),
                                                                                   (test_dir, testannot_dir)],
                                                                                  [(image_train_name_li, mask_train_name_li),
                                                                                   (image_val_name_li, mask_val_name_li),
                                                                                   (image_test_name_li, mask_test_name_li)]):
        save_full_path = os.path.join(save_dir, f_name)
        print('\nwriting :: \n{}'.format(save_full_path))
        with tf.io.gfile.GFile(save_full_path, mode='w') as f:
            with tqdm(total=len(image_file_names), position=1) as pbar:
                for image_file_name, mask_file_name in zip(image_file_names, mask_file_names):
                    image_line = os.path.join(image_dir, image_file_name)
                    mask_line = os.path.join(mask_dir, mask_file_name)
                    f.write(image_line + ' ' + mask_line+'\n')
                    pbar.update(1)


def generate_index_aihubpedestrian_segmentation(data_dir, save_dir):
    global SEED
    assert tf.io.gfile.isdir(save_dir), '{} is not a directory!'.format(save_dir)
    print('generate 3 files :\n{}\n{}\n{}\n'.format(os.path.join(save_dir, 'train.txt'),
                                                    os.path.join(save_dir, 'val.txt'),
                                                    os.path.join(save_dir, 'test.txt'),))
        
    surface_n_li = tf.io.gfile.listdir(data_dir)
    jpg_li = tf.io.gfile.glob(os.path.join(data_dir, '*/*.jpg'))
    png_li = tf.io.gfile.glob(os.path.join(data_dir, '*/MASK/*.png'))

    print('Number of image file {}'.format(len(jpg_li)))
    print('Number of mask file {}'.format(len(png_li)))

    images_fps = []
    masks_fps = []

    if len(jpg_li) == len(png_li):
        images_fps = jpg_li
        masks_fps = png_li
    else:
        print('Warning : Number of image file {} != number of mask file {}'.format(len(jpg_li), len(png_li)))
        print('Info : Check data manually.')
        with tqdm(total=len(surface_n_li), position=1) as pbar:
            for surface_n in surface_n_li:
                surface_n_dir = os.path.join(data_dir, surface_n)
                if tf.io.gfile.isdir(surface_n_dir):
                    # normal flow
                    f_name_li = tf.io.gfile.listdir(surface_n_dir)
                    for f_name in f_name_li:
                        f_path = os.path.join(surface_n_dir, f_name)
                        if not tf.io.gfile.isdir(f_path): 
                            # it is not mask dir
                            # image file or xml.. etc.
                            f_mask_path = os.path.join(surface_n_dir, 'MASK', f_name.split('.')[0]+'.png')
                            if os.path.exists(f_mask_path):
                                images_fps.append(f_path)
                                masks_fps.append(f_mask_path)
                            else:
                                pass
                        else:
                            pass
                else:
                    pass
                pbar.update(1)

    print('Number of image file {}'.format(len(images_fps)))
    print('Number of mask file {}'.format(len(masks_fps)))
    image_train_fps, image_test_fps, mask_train_fps, mask_test_fps = train_test_split(images_fps, masks_fps, 
                                                                                      test_size=1/10, shuffle=True, random_state=SEED)
    image_train_fps, image_val_fps, mask_train_fps, mask_val_fps = train_test_split(image_train_fps, mask_train_fps, 
                                                                                    test_size=1/9, shuffle=True, random_state=SEED)
    print('Number of train set : {}'.format(len(image_train_fps)))
    print('Number of val set : {}'.format(len(image_val_fps)))
    print('Number of test set : {}'.format(len(image_test_fps)))
    
    tf.io.gfile.makedirs(save_dir)
    for f_name, (images_fps, masks_fps) in zip(['train', 'val', 'test'], 
                                             [(image_train_fps, mask_train_fps),
                                              (image_val_fps, mask_val_fps),
                                              (image_test_fps, mask_test_fps)]):
        save_full_path = os.path.join(save_dir, f_name)
        print('\nwriting :: \n{}'.format(save_full_path))
        with tf.io.gfile.GFile(save_full_path, mode='w') as f:
            with tqdm(total=len(images_fps), position=1) as pbar:
                for image_line, mask_line in zip(images_fps, masks_fps):
                    f.write(image_line + ' ' + mask_line+'\n')
                    pbar.update(1)

    return images_fps, masks_fps

# implement your custom parser!


if DATASETNAME.lower() == 'camvid':
    generate_index_camvid_segmentation(DATASET_DIR_CAMVID,
                                       DATASET_TRAIN_X_DIR_CAMVID,
                                       DATASET_TRAIN_Y_DIR_CAMVID,
                                       DATASET_VAL_X_DIR_CAMVID,
                                       DATASET_VAL_Y_DIR_CAMVID,
                                       DATASET_TEST_X_DIR_CAMVID,
                                       DATASET_TEST_Y_DIR_CAMVID,
                                       TXT_FILE_DIR)
if DATASETNAME.lower() == 'aihubpedestrian':
    generate_index_aihubpedestrian_segmentation(DATASET_DIR_PEDESTRIAN, TXT_FILE_DIR)

generate 3 files :
/content/gdrive/MyDrive/data/index/CAMVID/train.txt
/content/gdrive/MyDrive/data/index/CAMVID/val.txt
/content/gdrive/MyDrive/data/index/CAMVID/test.txt

Number of image file 701
Number of mask file 701

writing :: 
/content/gdrive/MyDrive/data/index/CAMVID/train




writing :: 
/content/gdrive/MyDrive/data/index/CAMVID/val




writing :: 
/content/gdrive/MyDrive/data/index/CAMVID/test


### Debug

In [65]:
print('parsed : {}'.format(TXT_FILE_DIR))
if USE_GCS:
    !gsutil ls {TXT_FILE_DIR}
else:
    %cd {TXT_FILE_DIR}
    %ls -al

print('Read line example')
for name in ['train', 'val', 'test']:
    print('file {} example'.format(os.path.join(TXT_FILE_DIR, name)))
    with tf.io.gfile.GFile(os.path.join(TXT_FILE_DIR, name), mode='r') as f:
        for _ in range(5):
            line = f.readline()
            print(line.strip('\n'), '<line end!>')

parsed : /content/gdrive/MyDrive/data/index/CAMVID
/content/gdrive/MyDrive/data/index/CAMVID
total 85
-rw------- 1 root root 29001 Jul  2 03:26 test
-rw------- 1 root root 45467 Jul  2 03:26 train
-rw------- 1 root root 12019 Jul  2 03:26 val
Read line example
file /content/gdrive/MyDrive/data/index/CAMVID/train example
/content/gdrive/MyDrive/data/camvid/train/0001TP_006690.png /content/gdrive/MyDrive/data/camvid/trainannot/0001TP_006690.png <line end!>
/content/gdrive/MyDrive/data/camvid/train/0001TP_006720.png /content/gdrive/MyDrive/data/camvid/trainannot/0001TP_006720.png <line end!>
/content/gdrive/MyDrive/data/camvid/train/0001TP_006750.png /content/gdrive/MyDrive/data/camvid/trainannot/0001TP_006750.png <line end!>
/content/gdrive/MyDrive/data/camvid/train/0001TP_006780.png /content/gdrive/MyDrive/data/camvid/trainannot/0001TP_006780.png <line end!>
/content/gdrive/MyDrive/data/camvid/train/0001TP_006810.png /content/gdrive/MyDrive/data/camvid/trainannot/0001TP_006810.png <line

## Generate TFRecord file

### Hyper Parameters

In [136]:
if not TFRECORD_FROM_INDEXFILE:
    raise NotImplementedError

if TFRECORD_FROM_INDEXFILE:
    INDEX_FILE_DIR_FOR_TFRECORD = TXT_FILE_DIR #@param
    assert tf.io.gfile.exists(INDEX_FILE_DIR_FOR_TFRECORD), '{} not exists!'.format(INDEX_FILE_FULLPATH_FOR_TFRECORD)
    print('Source file : {}'.format(INDEX_FILE_DIR_FOR_TFRECORD))

ENCODING = 'png' #@param ["png", "jpeg"]

print("TFRecord Conversion result save at {}".format(TFRECORD_FILE_DIR))
print("See {}".format(TFRECORD_FILE_DIR))
if USE_GCS:
    !gsutil ls {TFRECORD_FILE_DIR}
else:
    !mkdir -p {TFRECORD_FILE_DIR}
    %cd {TFRECORD_FILE_DIR}
    %ls -al

Source file : gs://deer-rudolph/data/index/CAMVID
TFRecord Conversion result save at gs://deer-rudolph/data/tfrecord/CAMVID
See gs://deer-rudolph/data/tfrecord/CAMVID
gs://deer-rudolph/data/tfrecord/CAMVID/
gs://deer-rudolph/data/tfrecord/CAMVID/test
gs://deer-rudolph/data/tfrecord/CAMVID/train
gs://deer-rudolph/data/tfrecord/CAMVID/val


### Run

In [137]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _imread_tf(image_fp, mask_fp):
    image = tf.io.read_file(image_fp) # output : string tensor
    mask = tf.io.read_file(mask_fp) # output : string tensor
    image = tf.io.decode_png(image) # 3ch [h, w, 3], default datatype : tf.uint8
    mask = tf.io.decode_png(mask) # 1ch [h, w, 1] or 3ch [h, w, 3]
    #image = tf.io.convert_image_dtype(image, tf.uint8)
    #mask = tf.io.convert_image_dtype(mask, tf.uint8)
    return image, mask


def _image_feature(value):
    if ENCODING == 'png':
        return tf.train.Feature(bytes_list=tf.train.BytesList(
            value=[tf.io.encode_png(value).numpy()]))
    else:
        return tf.train.Feature(bytes_list=tf.train.BytesList(
            value=[tf.io.encode_jpeg(value).numpy()]))


def _bytes_feature(value): 
    return tf.train.Feature(bytes_list=tf.train.BytesList(
        value=[value.encode()]))


def _create_example(image, mask, image_fp=None, mask_fp=None):    
    feature = {
        "image": _image_feature(image),
        "mask": _image_feature(mask),
    }
    if (image_fp is not None) and (mask_fp is not None):
        feature["image_path"]=_bytes_feature(image_fp)
        feature["mask_path"]=_bytes_feature(mask_fp)
    return tf.train.Example(features=tf.train.Features(feature=feature))


def convert_to_tfrecord(images_fps, masks_fps, save_full_path):
    print('\nwriting :: \n{}'.format(save_full_path))
    with tqdm(total=len(images_fps)) as pbar:
        with tf.io.TFRecordWriter(save_full_path, options=None) as writer:
            for image_fp, mask_fp in zip(images_fps, masks_fps):
                image, mask = _imread_tf(image_fp, mask_fp)            
                record_bytes = _create_example(image, mask, image_fp, mask_fp).SerializeToString()
                writer.write(record_bytes)
                pbar.update(1)


def generate_tfrecord_camvid_segmentation(index_dir, save_dir, ignore_file_names=[]):
    index_file_name_li = tf.io.gfile.listdir(index_dir)
    images_fps = []
    masks_fps = []
    for index_file_name in index_file_name_li:
        reading_file = os.path.join(index_dir, index_file_name)
        print('\nreading :: \n{}'.format(reading_file))
        if tf.io.gfile.isdir(reading_file):
            print('This is a directory! Conversion passed.')
            continue
        if index_file_name in ignore_file_names:
            print('Ignored! Conversion passed.')
            continue
        with tf.io.gfile.GFile(reading_file, mode='r') as f:
            txt = f.readlines()
            for line in txt:
                line = line.strip('\n')
                image_fp, mask_fp = line.split(' ')
                images_fps.append(image_fp)
                masks_fps.append(mask_fp)
            convert_to_tfrecord(images_fps, masks_fps, os.path.join(save_dir, index_file_name))


def generate_tfrecord_aihubpedestrian_segmentation(index_dir, save_dir, ignore_file_names=[]):
    index_file_name_li = tf.io.gfile.listdir(index_dir)
    images_fps = []
    masks_fps = []
    for index_file_name in index_file_name_li:
        reading_file = os.path.join(index_dir, index_file_name)
        print('\nreading :: \n{}'.format(reading_file))
        if tf.io.gfile.isdir(reading_file):
            print('This is a directory! Conversion passed.')
            continue
        if index_file_name in ignore_file_names:
            print('Ignored! Conversion passed.')
            continue
        with tf.io.gfile.GFile(reading_file, mode='r') as f:
            txt = f.readlines()
            for line in txt:
                line = line.strip('\n')
                image_fp, mask_fp = line.split(' ')
                images_fps.append(image_fp)
                masks_fps.append(mask_fp)
            convert_to_tfrecord(images_fps, masks_fps, os.path.join(save_dir, index_file_name))

# implement your custom converter!


if not TFRECORD_FROM_INDEXFILE:
    raise NotImplementedError

if DATASETNAME.lower() == 'camvid':
    generate_tfrecord_camvid_segmentation(INDEX_FILE_DIR_FOR_TFRECORD, TFRECORD_FILE_DIR)
if DATASETNAME.lower() == 'aihubpedestrian':
    generate_tfrecord_aihubpedestrian_segmentation(INDEX_FILE_DIR_FOR_TFRECORD, TFRECORD_FILE_DIR, ignore_file_names=[])

# 참고 : arguemnt ignore_file_names 의 존재이유
# tfrecord 를 생성하는 일은 상당히 오래 걸린다. 그렇기 때문에 colab 을 항상 켜놓을 수 있는 상황이 아니라면, 일부 파일씩 돌려 가면서 변환하는 수밖에 없다.
# 또한, colab 은 시간제한이 최대 20시간정도이기 때문에 이것을 그대로 사용하면 무조건 timeout 이 나올수밖에 없다.
# 이 때 완전히 변환이 완료된 파일은 다시 생성하지 않도록 하기 위함이다.
# TODO : 지금은 single process 가 train test val 을 모두 처리하고 있는데, 이것을 multiprocess 로 바꾸면 좋을 것 같다.


reading :: 
gs://deer-rudolph/data/index/CAMVID/test

writing :: 
gs://deer-rudolph/data/tfrecord/CAMVID/test




reading :: 
gs://deer-rudolph/data/index/CAMVID/train

writing :: 
gs://deer-rudolph/data/tfrecord/CAMVID/train




reading :: 
gs://deer-rudolph/data/index/CAMVID/val

writing :: 
gs://deer-rudolph/data/tfrecord/CAMVID/val


### Debug

In [ ]:
print('parsed : {}'.format(TFRECORD_FILE_DIR))
if USE_GCS:
    !gsutil ls {TFRECORD_FILE_DIR}
else:
    %cd {TFRECORD_FILE_DIR}
    %ls -al


def _parse_tfrecord(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "mask": tf.io.FixedLenFeature([], tf.string),
        "image_path": tf.io.FixedLenFeature([], tf.string),
        "mask_path": tf.io.FixedLenFeature([], tf.string),
    }

    # case 1 : using parse_single_example parser
    # example = tf.io.parse_single_example(example, feature_description)

    # case 2 : using parse_example parser
    example = tf.io.parse_example(example, feature_description)

    # common
    example["image"] = tf.io.decode_png(example["image"], channels=3)
    example["mask"] = tf.io.decode_png(example["mask"])
    return example


for name in ['train', 'val', 'test']:
    reading_file = os.path.join(TFRECORD_FILE_DIR, name)
    print('reading :: {}'.format(reading_file))
    dataset = tf.data.TFRecordDataset(reading_file).map(_parse_tfrecord)

    for features in dataset.batch(1).take(1):
        for key in features.keys():
            print('there is key named : {}'.format(key))
        
        ##debugging code
        plt.figure(figsize=(14, 14))
        plt.subplot(1,6, 1)
        plt.imshow(features["image"].numpy().squeeze())
        plt.title('TFRecord')

        plt.subplot(1,6,2)
        plt.imshow((features["mask"].numpy().squeeze()/(features["mask"].numpy().max())) , cmap='gray')
        plt.title('TFRecord')
        print('from tfrecord : {}'.format(np.unique((features["mask"].numpy().squeeze()[270:,:70]))))
    
        ##debugging code
        if not USE_GCS:
            plt.subplot(1,6,3)
            p_image = features["image_path"].numpy()[0].decode("utf-8")
            cv_image = cv2.imread(p_image)
            cv_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
            plt.imshow(cv_image)
            plt.title('cv.imread')
                
            plt.subplot(1,6,4)
            p_mask = features["mask_path"].numpy()[0].decode("utf-8")
            cv_mask = cv2.imread(p_mask)
            plt.imshow((cv_mask/cv_mask.max())[270:,:70], cmap='gray')
            plt.title('cv.imread')
            print('from cv.imread : {}'.format(np.unique(temp_mask[270:,:70])))

        ##debugging code
        tf_image, tf_mask = _imread_tf(p_image, p_mask)
        plt.subplot(1,6,5)
        plt.title('tf.io')
        plt.imshow(tf_image.numpy().squeeze())
        
        ##debugging code
        plt.subplot(1,6,6)
        plt.title('tf.io')
        plt.imshow((tf_mask.numpy().squeeze()/tf_mask.numpy().max())[270:,:70], cmap='gray')
        print('from tf.io : {}'.format(np.unique(temp_mask[270:,:70])))

        ##debugging code
        # Three result should be same.
        print('cnt of True, tfrecord == tf.io : {}'.format(np.count_nonzero(features["image"].numpy().squeeze()==tf_image) ))
        if not USE_GCS:
            print('cnt of True, tfrecord == cv.imread : {}'.format(np.count_nonzero(features["image"].numpy().squeeze()==cv_image) ))
            print('cnt of True, tf.io == cv.imread : {}'.format(np.count_nonzero(cv_image==tf_image) ))

# debug

In [ ]:
def filter_args(*args):
    print(args)
    return args
def filter_kwargs(**kwargs):
    print(kwargs)
    kwargs['byebye'] = 'hahah'
    return kwargs
def temp(*args, **kwargs):
    filtered_args = filter_args(*args)
    filtered_kwargs = filter_kwargs(**kwargs)
    print(filtered_args, filtered_kwargs)

temp('hi', hey="you!", geogi='neo')

('hi',)
{'hey': 'you!', 'geogi': 'neo'}
('hi',) {'hey': 'you!', 'geogi': 'neo', 'byebye': 'hahah'}


In [33]:
name_li = ['train', 'val', 'test']
for name in name_li:
    with tf.io.gfile.GFile(os.path.join(TXT_FILE_DIR, name), mode='r') as f:
        print('read : {}'.format(name))
        for _ in range(2):
            line = f.readline()
            print(line.strip('\n'), '<line end!>')

read : train
/content/gdrive/MyDrive/data/camvid/train/0001TP_006690.png /content/gdrive/MyDrive/data/camvid/trainannot/0001TP_006690.png <line end!>
/content/gdrive/MyDrive/data/camvid/train/0001TP_006720.png /content/gdrive/MyDrive/data/camvid/trainannot/0001TP_006720.png <line end!>
read : val
/content/gdrive/MyDrive/data/camvid/val/0016E5_07959.png /content/gdrive/MyDrive/data/camvid/valannot/0016E5_07959.png <line end!>
/content/gdrive/MyDrive/data/camvid/val/0016E5_07961.png /content/gdrive/MyDrive/data/camvid/valannot/0016E5_07961.png <line end!>
read : test
/content/gdrive/MyDrive/data/camvid/test/0001TP_008550.png /content/gdrive/MyDrive/data/camvid/testannot/0001TP_008550.png <line end!>
/content/gdrive/MyDrive/data/camvid/test/0001TP_008580.png /content/gdrive/MyDrive/data/camvid/testannot/0001TP_008580.png <line end!>
